In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Deep learning school/HW6/packages')

import dataset
import models
import torch
import torchvision
from torchvision import transforms
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

Mounted at /content/drive


In [2]:
!unzip -q "/content/drive/MyDrive/Deep learning school/HW6/real2jojo_test.zip"

In [3]:
from matplotlib.pyplot import imshow
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset_real= dataset.FolderDataSet("./real2jojo_test/real", transform=transform)
dataloader_real = torch.utils.data.DataLoader(dataset_real, batch_size = 1)
dataset_jojo = dataset.FolderDataSet("./real2jojo_test/jojo", transform=transform)
dataloader_jojo = torch.utils.data.DataLoader(dataset_jojo, batch_size = 1)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

pool_images = None
pool_targets = None
gan = models.CycleGAN(lambd = 1, resnet_layers = 9, device = device, buffer_images=pool_images, dim_gen = 64, buffer_targets=pool_targets, dropout = 0.5)
gan.to(device)
checkpoint = torch.load('/content/drive/MyDrive/Deep learning school/HW6/real2jojo_weights')
gan.load_state_dict(checkpoint['gan_state_dict'])

/content/drive/MyDrive/Deep learning school/HW6/packages/models.py:9: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(x.weight, m, s)


<All keys matched successfully>

In [4]:
from PIL import Image  
import PIL
gan.eval()
resize = False
scale = 512
for i, img_real in enumerate(dataloader_real):
    img_real = img_real.to(device)
    if resize:
        old_size = img_real.shape[2:]
        if old_size[0] < old_size[1]:
            transform = transforms.Resize((scale, int(scale*old_size[1]/old_size[0])))
        else:
            transform = transforms.Resize((int(scale*old_size[0]/old_size[1]), scale))
        
        img_real = transform(img_real)
        transform_inv = transforms.Resize(old_size)
    
    converted_real = torchvision.transforms.ToPILImage()(dataset.de_norm(gan(img_real)[0].cpu()))
    if resize:
        converted_real = transform_inv(converted_real)
    converted_real.save('/content/drive/MyDrive/Deep learning school/HW6/real2jojo_results/real2jojo/' + str(i) + '.png')

for i, img_jojo in enumerate(dataloader_jojo):
    img_jojo = img_jojo.to(device)
    converted_jojo = torchvision.transforms.ToPILImage()(dataset.de_norm(gan(img_jojo, inverse = True)[0].cpu()))
    converted_jojo.save('/content/drive/MyDrive/Deep learning school/HW6/real2jojo_results/jojo2real/' + str(i) + '.png')